## 데이터 전처리 및 저장

In [22]:
# 데이터 불러오기
import pandas as pd
import numpy as np 
import os

train = pd.read_csv("랜포train.CSV")
train.head()


,season,temperature,cloudy,rain,sunny,airpolution,humid(%),품목 분류,cluster,menu
0,fall,-0.6,0,0,1,23.0,60.9,라떼,3,아이스 그린티라떼
1,fall,-0.6,0,0,1,23.0,60.9,커피,1,아메리카노
2,fall,-0.6,0,0,1,23.0,60.9,커피,1,아이스 아메리카노
3,fall,-0.6,0,0,1,23.0,60.9,커피,5,카페라떼
4,fall,-0.6,0,0,1,23.0,60.9,커피,3,아이스 카페라떼


In [23]:
drop_index = train[(train["menu"] == '아이스 아메리카노')|(train["menu"] == '아이스 카페라떼')|(train["menu"] == '아이스 바닐라라떼')|(train["menu"] == '카페라떼')|(train["menu"] == '아메리카노')].index
#아이스 아메리카노, 아이스 카페라떼, 아이스 바닐라라떼, 아메리카노

print(drop_index)

Int64Index([   1,    2,    3,    4,    7,   15,   16,   17,   18,   20,
            ...
            4855, 4856, 4857, 4859, 4861, 4880, 4882, 4883, 4886, 4887],
           dtype='int64', length=1211)


In [24]:
train.drop(drop_index, inplace=True)

In [25]:
print(train.shape)

(3690, 10)


In [26]:
# 메뉴 칼럼 드랍, season더미변수화 진행

train_dummies_season = pd.DataFrame(pd.get_dummies(train['season']))

In [30]:
train_final = pd.concat([train.iloc[:,1:7], train_dummies_season, train[['cluster']]], axis=1)

In [53]:
# index수정
train_final.reset_index(inplace = True)
train_final.index=train_final.index + 1
train_final.drop(columns= "index", inplace=True)
train_final.head()

,temperature,cloudy,rain,sunny,airpolution,humid(%),fall,spring,summer,winter,cluster
1,-0.6,0,0,1,23.0,60.9,1,0,0,0,3
2,-0.6,0,0,1,23.0,60.9,1,0,0,0,5
3,-0.6,0,0,1,23.0,60.9,1,0,0,0,5
4,-0.6,0,0,1,23.0,60.9,1,0,0,0,5
5,-0.6,0,0,1,23.0,60.9,1,0,0,0,4


In [57]:
x = train_final.iloc[:,:-1].values
y = train_final.iloc[:,-1:].values

## kfold, random forest

In [58]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

# split 개수, 셔플 여부 및 seed 설정
str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 50)

# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in str_kf.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)  # 모델 학습
    
    y_pred = model.predict(X_test)  # 예측 수행
    
    accuracy = accuracy_score(y_test, y_pred)  # 정확도 계산
    print(f"Accuracy: {accuracy}")



C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\248147687.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)  # 모델 학습


Accuracy: 0.2222222222222222


C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\248147687.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)  # 모델 학습


Accuracy: 0.22357723577235772


C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\248147687.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)  # 모델 학습


Accuracy: 0.24390243902439024


C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\248147687.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)  # 모델 학습


Accuracy: 0.23577235772357724


C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\248147687.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)  # 모델 학습


Accuracy: 0.26151761517615174


In [62]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

# 모델 훈련함수
def train_rf_model_with_kfold(x, y, k):
    kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 2019100378)
    model = RandomForestClassifier()
    
    for train_index, _ in kfold.split(x, y):
        X_train, y_train = x[train_index], y[train_index]
        model.fit(X_train, y_train)

    return model
    


In [64]:
from sklearn.metrics import accuracy_score

# 정확도 반환 함수
def get_accuracy(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy


In [63]:
from sklearn.model_selection import train_test_split
x = train_final.iloc[:,:-1].values
y = train_final.iloc[:,-1:].values

# train이랑 valid 나누기
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y, random_state=1004)


rf_model = train_rf_model_with_kfold(x_train, y_train, 5)



C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\3339543208.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\3339543208.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\3339543208.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\3339543208.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
C:\Users\hop09\AppData\Local\Temp\ipykernel_17816\3339543208.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_trai

In [65]:
y_pred = rf_model.predict(x_valid)
accuracy = get_accuracy(y_valid, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.21544715447154472


In [67]:
print(x_valid.shape, y_valid.shape, y_pred.shape)


(738, 10) (738, 1) (738,)


In [68]:
print(x_valid)

[[24.6  0.   0.  ...  0.   0.   0. ]
 [29.4  0.   1.  ...  0.   1.   0. ]
 [ 2.9  0.   1.  ...  0.   0.   0. ]
 ...
 [20.9  0.   0.  ...  0.   0.   0. ]
 [21.9  0.   1.  ...  1.   0.   0. ]
 [-3.   0.   1.  ...  0.   0.   1. ]]


In [69]:
train_final.head()

,temperature,cloudy,rain,sunny,airpolution,humid(%),fall,spring,summer,winter,cluster
1,-0.6,0,0,1,23.0,60.9,1,0,0,0,3
2,-0.6,0,0,1,23.0,60.9,1,0,0,0,5
3,-0.6,0,0,1,23.0,60.9,1,0,0,0,5
4,-0.6,0,0,1,23.0,60.9,1,0,0,0,5
5,-0.6,0,0,1,23.0,60.9,1,0,0,0,4


In [72]:
# 예측결과 데이터프레임화
import pandas as pd

# 데이터프레임 생성
column_names = ['temperature',	'cloudy',	'rain',	'sunny',	'airpolution',	'humid(%)',	'fall',	'spring'	,'summer',	'winter']
df_x = pd.DataFrame(x_valid, columns=column_names)

y_name = ['real_cluster']
df_real_y = pd.DataFrame(y_valid, columns = y_name)

y_name2 = ['pred_cluster']
df_pred_y = pd.DataFrame(y_pred, columns=y_name2)

model_result = pd.concat([df_x, df_real_y, df_pred_y], axis=1)
model_result.head()

# 엑셀 저장
model_result.to_excel("모델결과반환.xlsx", encoding='cp949')

c:\Users\hop09\Desktop\anaconda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
